In [2]:
from music21 import *

In [ ]:
def harmonize(melody):
    # > let's at first assume we're assigning a chord every beat for simplicity's sake
    
    # analyze_key(melody)
    
    # chord[last melody note] = tonic
    
    # for beat in melody[::-1]:      (starting with last chord and working backwards)
        # if following chord is tonic & melody note in V7 chord:
            # chord[beat] = V7
        # elif following chord is tonic & melody note in IV (or ii) chord:
            # chord[beat] = IV (or ii)
        # elif following chord is major & melody note in that chord:
            # chord[beat] = relative minor
        # elif V7 of following chord includes melody note:
            # chord[beat] = V7 of following chord (work backwards around circle of fourths)
        # else:
            # chord[beat] = closest chord (re: circle of fourths/fifths) that includes the melody note